# Import libraries

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

/Users/DRRRR/anaconda/envs/machinelearning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/DRRRR/anaconda/envs/machinelearning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/DRRRR/anaconda/envs/machinelearning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/DRRRR/anaconda/

In [2]:
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


# Processing Images

In [3]:
# opens images and resizes them
def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize((512, round(512/img.size[0]*img.size[1])))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

In [4]:
# loads images and runs them through vgg19 preprocessing
def process_image(image_path):
    img = load_image(image_path)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return img

In [5]:
# deprocess images from vgg19
def deprocess_image(image):
    image = np.squeeze(image, axis = 0)
    mean = [103.939, 116.779, 123.68]
    image[:, :, 0] += 103.939
    image[:, :, 1] += 116.779
    image[:, :, 2] += 123.68
    image = image[:, :, ::-1]

    image = np.clip(image, 0, 255).astype('uint8')
    return image

# Loading Model

In [6]:
from tensorflow.python.keras import models

In [7]:
# layers used for feature extraction

s_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

c_layers = ['block5_conv2']

In [8]:
# create vgg19 model
def get_model():
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False

    style_layers = [vgg.get_layer(name).output for name in s_layers]
    content_layers = [vgg.get_layer(name).output for name in c_layers]

    model_layers = style_layers + content_layers

    model = models.Model(vgg.input, model_layers)

    return model

# Gram Matrix

In [9]:
def gram_matrix(tensor):
    channels = int(tensor.shape[-1])
    a = tf.reshape(tensor, [-1, channels])
    gram = tf.matmul(a, a, transpose_a = True)
    return gram

# Extract Feature Representation

In [10]:
def get_content_features(content, model):
    content_output = model(content)
    content_features = [layer[0] for layer in content_output[len(s_layers):]]
    return content_features

In [11]:
def get_style_features(style, model):
    style_output = model(style)
    style_features = [layer[0] for layer in style_output[:len(s_layers)]]
    return style_features

# Loss

In [12]:
def get_style_loss(image, gram_style, style_weight, model):
    style_weights = [0.5, 0.3, 0.1, 0.075, 0.025]

    style_image_features = get_style_features(image, model)

    style_loss = 0

    for i in range(len(style_weights)):
        style_loss += style_weights[i] * tf.reduce_mean(tf.square(gram_matrix(style_image_features[i]) - gram_style[i]))

    return style_loss * style_weight

In [13]:
def get_content_loss(image, content_features, content_weight, model):
    content_image_features = get_content_features(image, model)

    content_loss = tf.reduce_mean(tf.square(content_image_features[0] - content_features[0]))

    return content_loss * content_weight

# Graidents

In [14]:
def compute_grad(content_features, style_features, gram_style, image, content_weight, style_weight, model):
    with tf.GradientTape() as tape:
        tape.watch(image)

        style_loss = get_style_loss(image, gram_style, style_weight, model)
        content_loss = get_content_loss(image, content_features, content_weight, model)
    
        loss = style_loss + content_loss

    grad = tape.gradient(loss, image)

    return grad, loss

# Training

In [15]:
def train(content_path, style_path, steps=1000,content_weight=1e3, style_weight=1e-2):
    model = get_model()
    for layer in model.layers:
        layer.trainable = False
    
    content = process_image(content_path)
    style = process_image(style_path)
    
    image = process_image(content_path)
    image = tf.Variable(image)
    
    content_features = get_content_features(content, model)
    print("step4")
    style_features = get_style_features(style, model)
    print("step5")
    gram_style = [gram_matrix(style_feature) for style_feature in style_features]
    
    optimizer = tf.keras.optimizers.Adam(learning_rate = 5)
    
    img = []
    best_loss = float('inf')
    
    print("Starting...")
    for i in range(steps):
        grad, loss = compute_grad(content_features, style_features, gram_style, image, content_weight, style_weight, model)
        optimizer.apply_gradients([(grad, image)])

    if loss < best_loss:
        best_loss = loss
        best_image = deprocess_image(image.numpy())

    if i % 100 == 0:
        img.append(deprocess_image(image.numpy()))
        print("step#: ", i, "/", steps)
        print("total loss:", loss)
  
    return best_image, img

# Running Everything

In [16]:
# download images and place them in My Drive
# mount drive under files on the left
content_path = 'content_beach.jpeg'
style_path = 'style_monetgreen.jpeg'

In [17]:
final, imgs = train(content_path, style_path)

Instructions for updating:
Colocations handled automatically by placer.


ValueError: Dimensions must be equal, but are 4 and 3 for 'model/block1_conv1/Conv2D' (op: 'Conv2D') with input shapes: [1,369,512,4], [3,3,3,64].

In [ ]:
plt.imshow(final)

In [ ]:
fig, axs = plt.subplots(int((len(imgs) / 5)), 5)
for ax, image in zip(axs.flatten(), imgs):
  ax.imshow(image)
  ax.axis("off")

In [ ]:
plt.axis('off')
plt.imshow(final)
plt.savefig('StyleTransfer.png', bbox_inches='tight', pad_inches=0)

In [ ]:
files.download('StyleTransfer.png')